In [ ]:
!pip install pyspark

In [ ]:
#Pour lire un fichier CSV et le charger dans un DataFrame Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL Students").getOrCreate()

# Charger le fichier CSV
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/etudiants_large.csv", header=True, inferSchema=True)
df.show()

In [ ]:
from pyspark.sql.functions import upper, col  # Import upper and col functions

# Convertir les noms en majuscules
df1 = df.withColumn("nom", upper(col("nom")))
df1.show()

+--------+---------+---+------------+
|     nom|   prenom|age|     filiere|
+--------+---------+---+------------+
|  BROOKS|    Tracy| 23|   Marketing|
|   RAMOS|     John| 23|     Gestion|
|MITCHELL|  Kristin| 20|Informatique|
|  CHAVEZ|    Kevin| 26|Data Science|
|BAUTISTA|    James| 26|     Gestion|
| BALDWIN|   Justin| 22|Data Science|
|MARTINEZ|      Ray| 18|   Marketing|
|   SCOTT|  Zachary| 21|     Gestion|
|  DANIEL|   Alexis| 23|   Marketing|
|CRAWFORD|     Erin| 25|   Marketing|
|    HALL|    Glenn| 26|Informatique|
| WOODARD|Christina| 22|   Marketing|
| JOHNSON| Jonathan| 20|Informatique|
|  NORTON|   Amanda| 23|     Gestion|
|   JONES|Elizabeth| 26|Data Science|
|  LARSON| Jennifer| 23|   Marketing|
|MARTINEZ|    Peter| 25|Informatique|
|    BOYD|  Bridget| 26|Informatique|
|    CHEN|   Ashley| 20|   Marketing|
|    WOOD|   Daniel| 22|     Gestion|
+--------+---------+---+------------+
only showing top 20 rows



In [ ]:
#Filtrer les étudiants de plus de 20 ans
df2= df1.filter(df1["age"] > 20)
df2.show()

+--------+-----------+---+------------+
|     nom|     prenom|age|     filiere|
+--------+-----------+---+------------+
|  BROOKS|      Tracy| 23|   Marketing|
|   RAMOS|       John| 23|     Gestion|
|  CHAVEZ|      Kevin| 26|Data Science|
|BAUTISTA|      James| 26|     Gestion|
| BALDWIN|     Justin| 22|Data Science|
|   SCOTT|    Zachary| 21|     Gestion|
|  DANIEL|     Alexis| 23|   Marketing|
|CRAWFORD|       Erin| 25|   Marketing|
|    HALL|      Glenn| 26|Informatique|
| WOODARD|  Christina| 22|   Marketing|
|  NORTON|     Amanda| 23|     Gestion|
|   JONES|  Elizabeth| 26|Data Science|
|  LARSON|   Jennifer| 23|   Marketing|
|MARTINEZ|      Peter| 25|Informatique|
|    BOYD|    Bridget| 26|Informatique|
|    WOOD|     Daniel| 22|     Gestion|
|  RITTER|Christopher| 22|   Marketing|
|    ROSE|       Tina| 21|   Marketing|
|   WELLS|    Jessica| 21|Data Science|
| WHEELER|     Amanda| 24|     Gestion|
+--------+-----------+---+------------+
only showing top 20 rows



In [ ]:
#Calculer la moyenne d'âge par filière
from pyspark.sql.functions import avg
stats = df2.groupBy("Filiere").agg(avg("age").alias("moyenne_age"))
stats.show()

+------------+------------------+
|     Filiere|       moyenne_age|
+------------+------------------+
|Informatique|24.384615384615383|
|     Gestion|23.333333333333332|
|   Marketing|22.904761904761905|
|Data Science|             23.25|
+------------+------------------+



In [ ]:
#Sauvegarde les données transformées dans un fichier CSV
df2.write.csv("/content/etudiants_transforme.csv", header=True)

In [ ]:
!pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
import csv
import json
from kafka import KafkaProducer

# Configuration du producteur Kafka
producer = KafkaProducer(
    bootstrap_servers='kafka-fadbfb9-hyibtissam-ca1b.h.aivencloud.com:11468',
    security_protocol='SSL',
    ssl_cafile='/content/drive/MyDrive/Colab Notebooks/exam_BI/ca.pem',
    ssl_certfile='/content/drive/MyDrive/Colab Notebooks/exam_BI/service.cert',
    ssl_keyfile='/content/drive/MyDrive/Colab Notebooks/exam_BI/service.key',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')  # Convertir les données en JSON
)

# Chemin du fichier CSV
csv_file_path = "/content/drive/MyDrive/Colab Notebooks/exam_BI/etudiants_large.csv"

# Lire le fichier CSV et envoyer chaque ligne à Kafka
with open(csv_file_path, mode='r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)  # Lire les lignes sous forme de dictionnaire
    for row in csv_reader:
        producer.send('ibtissam', value=row)  # Envoyer la ligne comme message JSON
        print(f"Message envoyé : {row}")

# Finaliser l'envoi
producer.flush()
print("Tous les messages ont été envoyés !")

Message envoyé : {'nom': 'Brooks', 'prenom': 'Tracy', 'age': '23', 'filiere': 'Marketing'}
Message envoyé : {'nom': 'Ramos', 'prenom': 'John', 'age': '23', 'filiere': 'Gestion'}
Message envoyé : {'nom': 'Mitchell', 'prenom': 'Kristin', 'age': '20', 'filiere': 'Informatique'}
Message envoyé : {'nom': 'Chavez', 'prenom': 'Kevin', 'age': '26', 'filiere': 'Data Science'}
Message envoyé : {'nom': 'Bautista', 'prenom': 'James', 'age': '26', 'filiere': 'Gestion'}
Message envoyé : {'nom': 'Baldwin', 'prenom': 'Justin', 'age': '22', 'filiere': 'Data Science'}
Message envoyé : {'nom': 'Martinez', 'prenom': 'Ray', 'age': '18', 'filiere': 'Marketing'}
Message envoyé : {'nom': 'Scott', 'prenom': 'Zachary', 'age': '21', 'filiere': 'Gestion'}
Message envoyé : {'nom': 'Daniel', 'prenom': 'Alexis', 'age': '23', 'filiere': 'Marketing'}
Message envoyé : {'nom': 'Crawford', 'prenom': 'Erin', 'age': '25', 'filiere': 'Marketing'}
Message envoyé : {'nom': 'Hall', 'prenom': 'Glenn', 'age': '26', 'filiere': 'I

In [ ]:
from kafka import KafkaConsumer
import json

# Configuration du consommateur Kafka
TOPIC_NAME = "ibtissam"
consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers='kafka-fadbfb9-hyibtissam-ca1b.h.aivencloud.com:11468',
    client_id="CONSUMER_CLIENT_ID",
    group_id="CONSUMER_GROUP_ID",
    security_protocol='SSL',
    ssl_cafile='/content/drive/MyDrive/Colab Notebooks/exam_BI/ca.pem',
    ssl_certfile='/content/drive/MyDrive/Colab Notebooks/exam_BI/service.cert',
    ssl_keyfile='/content/drive/MyDrive/Colab Notebooks/exam_BI/service.key',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))  # Désérialisation des messages JSON
)

# Consommer les messages et les afficher
print("En attente de messages...")

for message in consumer:
    # Décodage et affichage du message
    print(f"Message reçu : {message.value}")



En attente de messages...


KeyboardInterrupt: 